In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Constraints analysis

### 1 Example setup, A and b plots

In [ ]:
from angle_set import AngleSet

d = 2 # do not change.
N = 5

np.random.seed(51)

angle_set = AngleSet(N=N, d=d)
angle_set.set_points(mode='random')
angle_set.plot_all()

In [ ]:
Apoly, bpoly = angle_set.get_polygon_constraints([3, 4])
assert np.allclose(Apoly.dot(angle_set.theta), bpoly)
plt.matshow(np.c_[Apoly, bpoly.reshape((-1, 1))])
plt.title('polygon constraints')

In [ ]:
Arays, brays = angle_set.get_ray_constraints(verbose=False)
assert np.allclose(Arays.dot(angle_set.theta), brays)
plt.matshow(np.c_[Arays, brays.reshape((-1, 1))])
plt.title('single constraints')

In [ ]:
from angle_set import combine_fullrank_matrix

Afull, bfull = combine_fullrank_matrix(Arays, Apoly, brays, bpoly, False)
print('kept rows: {} of {}'.format(Afull.shape[0] - Arays.shape[0], Apoly.shape[0]))
plt.matshow(np.c_[Afull, bfull])
plt.title('full rank matrix')

In [ ]:
from angle_set import get_numbers

n_rays, n_poly = get_numbers(angle_set.N)

assert n_rays == Arays.shape[0]
print('number of ray constraints found:', n_rays)

assert n_poly == Afull.shape[0] - Arays.shape[0]
print('number of poly constraints found:', n_poly)

### 2. Compare obtained number of constraints

In [ ]:
from helpers import savefig
import seaborn as sns

def plot_here(ax, range_):
    colors = sns.color_palette('tab20')
    ax.stackplot(Ns[range_], 
                 [n_rays_list[range_], n_poly_list[range_], n_nonlinear_list[range_], DOFs[range_]], 
                 labels=['single', 'triangle', 'non-linear', 'DOF'], 
                 colors=colors)
    ax.plot(Ns[range_], Ms[range_], 
            color='black', label='no. angles $M$', ls=':')
    ax.set_xlim(Ns[range_][0], Ns[range_][-1])
    
def plot_inset(ax, loc=[0.5, 0.5, 0.47, 0.47], range_full=[0, 10],
               range_inset = [14, 15, 1000, 1500]):
    """ Create zoom inside the plot. """
    axins = ax.inset_axes(loc) # location (x,y), size
    plot_here(axins, range_full)
    # sub region of the original image
    x1, x2, y1, y2 = range_inset
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels('')
    axins.set_yticklabels('')
    axins.set_xticks([])
    axins.set_yticks([])
    axins.set_facecolor((1, 1, 1, 0.5))
    r_, lines_ = ax.indicate_inset_zoom(axins, label=None)
    [l.set_color('black') for l in lines_]
    [l.set_alpha(1.0) for l in lines_]
    r_.set_alpha(1.0)
    r_.set_edgecolor('black')

In [ ]:
d = 2 # do not change.

print('N \t M \t DOF \t necessary \t linear \t nonlinear')

Ns = np.arange(4, 20)
Ms = np.empty((len(Ns),))
DOFs = np.empty((len(Ns),))
n_linear_list = np.empty((len(Ns),))
n_nonlinear_list = np.empty((len(Ns),))
n_poly_list = np.empty((len(Ns),))
n_rays_list = np.empty((len(Ns),))

for i, N in enumerate(Ns):
    angle_set = AngleSet(N=N, d=d)
    angle_set.set_points(mode='random')
    n_rays = angle_set.get_n_rays() 
    n_poly = angle_set.get_n_poly()

    Apoly, bpoly = angle_set.get_polygon_constraints([3])
    Aray, bray = angle_set.get_ray_constraints()
    
    # Sanity check only. is taking too long for N >= 11
    if N < 11:
        Afull, bfull = combine_fullrank_matrix(Aray, Apoly, bray.flatten(), bpoly, False)
        assert n_rays == Aray.shape[0]
        assert n_poly == Afull.shape[0] - Aray.shape[0]
    
    Afull = np.vstack([Aray, Apoly[:n_poly]])
    bfull = np.hstack([bray, bpoly[:n_poly]])
    
    #DOF = angle_set.N * angle_set.d - 2*angle_set.d - 1
    DOF = angle_set.get_DOF()
    necessary = angle_set.M - DOF
    
    nonlinear = sum([1] + [i for i in range(2, angle_set.N-2)])
    assert nonlinear == angle_set.get_n_sine()
    
    assert nonlinear + n_rays + n_poly == necessary
    print('{} \t {} \t {} \t {} \t {} \t {}'.format(
        angle_set.N, angle_set.M,  DOF, necessary, n_rays + n_poly, nonlinear
    ))
    
    n_poly_list[i] = n_poly
    n_rays_list[i] = n_rays
    n_linear_list[i] = n_rays + n_poly
    n_nonlinear_list[i] = nonlinear
    Ms[i] = angle_set.M
    DOFs[i] = DOF

### 3 Create plot for paper

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(6, 2.5)
range_ = np.arange(14)
plot_here(ax, range_)
plot_inset(ax, 
           loc=[0.02, 0.25, 0.5, 0.35], 
           range_full=range_,
           range_inset=[4, 8, 0, 200])
h, l = ax.get_legend_handles_labels()
ax.legend(h[::-1], l[::-1], loc='upper left',ncol=3)
ax.set_ylabel('count', fontsize=12)
ax.set_xlabel('number of points $N$', fontsize=12)
plt.tight_layout()
ax.grid()

#savefig(fname='plots/number_constraints.pdf')